# **LOADING MAP DATA**

In [2]:
import mysql.connector

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='phonepe'
)

cursor = conn.cursor()


In [3]:
import os
import json

def load_map_insurance_hover(base_path, cursor, conn):
    for state in os.listdir(base_path):
        state_path = os.path.join(base_path, state)
        if not os.path.isdir(state_path):
            continue

        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            for quarter_file in os.listdir(year_path):
                if not quarter_file.endswith(".json"):
                    continue

                quarter = int(quarter_file.strip("q.json"))
                file_path = os.path.join(year_path, quarter_file)

                with open(file_path, "r") as f:
                    data = json.load(f)

                try:
                    hover_list = data["data"]["hoverDataList"]
                except KeyError:
                    continue

                for item in hover_list:
                    district = item["name"]
                    metric = item["metric"][0]
                    count = metric.get("count", 0)
                    amount = metric.get("amount", 0)

                    cursor.execute("""
                        INSERT INTO map_insurance (year, quarter, state, district, insurance_count, insurance_amount)
                        VALUES (%s, %s, %s, %s, %s, %s)
                    """, (int(year), int(quarter), state, district, count, amount))

    conn.commit()
    print("map_insurance data from hover inserted.")


In [4]:
base_path = r"C:\Users\sharan\Documents\phonepe_transaction_insights\data\pulse-master\pulse-master\data\map\insurance\hover\country\india\state"
load_map_insurance_hover(base_path, cursor, conn)


map_insurance data from hover inserted.
